In [1]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

SEED = 43
np.random.seed(SEED)

Using TensorFlow backend.


In [2]:
df_pos = pd.read_csv(open('clean_positive_train.csv','r'), encoding='utf-8', engine='c')
df_neg = pd.read_csv(open('clean_negative_train.csv','r'), encoding='utf-8', engine='c')

df_pos['text'] = df_pos['text'].astype(str)
df_pos['parent_text'] = df_pos['parent_text'].astype(str)

df_neg['text'] = df_neg['text'].astype(str)
df_neg['parent_text'] = df_neg['parent_text'].astype(str)

In [3]:
df_pos.describe()

,score,ups,controversiality,parent_score,parent_ups,parent_controversiality
count,99999.0,99999.000000,99999.000000,99999.000000,99999.000000,99999.00000
mean,1.0,198.155082,0.000020,0.990770,368.973070,0.00064
std,0.0,256.334734,0.004472,0.095629,535.679712,0.02529
min,1.0,66.000000,0.000000,0.000000,-8907.000000,0.00000
25%,1.0,83.000000,0.000000,1.000000,84.000000,0.00000
50%,1.0,116.000000,0.000000,1.000000,184.000000,0.00000
75%,1.0,200.000000,0.000000,1.000000,417.000000,0.00000
max,1.0,5488.000000,1.000000,1.000000,9531.000000,1.00000


In [4]:
df_neg.describe()

,score,ups,controversiality,parent_score,parent_ups,parent_controversiality
count,99998.0,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000
mean,0.0,-14.583632,0.001490,0.912518,67.440239,0.002720
std,0.0,15.649932,0.038572,0.282541,219.047635,0.052083
min,0.0,-1077.000000,0.000000,0.000000,-1622.000000,0.000000
25%,0.0,-15.000000,0.000000,1.000000,6.000000,0.000000
50%,0.0,-10.000000,0.000000,1.000000,15.000000,0.000000
75%,0.0,-8.000000,0.000000,1.000000,44.000000,0.000000
max,0.0,-6.000000,1.000000,1.000000,14776.000000,1.000000


In [5]:
df = pd.concat([df_pos, df_neg])
df = df.sample(frac=1).reset_index(drop=True)
df.dropna(axis=0, inplace=True)
df['combined'] = df[['text', 'parent_text']].apply(lambda x: ' '.join(x), axis=1)

X = df['combined']
y = df['score']
#X_train, X_test, y_train, y_test = train_test_split(text_data,text_score, test_size = 0.20, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=SEED)

# To be sure we don't use indices to predict something
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

X_train: (189997,)
X_test: (10000,)
y_train: (189997,)
y_test: (10000,)


In [7]:
# Токенизируем текст
tokenizer = Tokenizer(num_words=10000, lower=True, split=' ', document_count=0)
tokenizer.fit_on_texts(X)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [8]:
#каждое предложение имеет разную длину, и мы хотим каждый раз передавать один и тот же вектор длины в нашу нейронную сеть,
#мы дополняем их, добавляя нули в конце каждой последовательности, поэтому каждое из них имеет длину 128 целых чисел.
train_data = keras.preprocessing.sequence.pad_sequences(X_train_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=128)

test_data = keras.preprocessing.sequence.pad_sequences(X_test_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=128)

In [9]:
print(len(train_data[0]), len(train_data[1]))

128 128


In [10]:
#делаем нейросеть
vocab_size = len(tokenizer.word_index) + 1

model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 4))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(2, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

W0405 14:16:20.825892 140362287126336 deprecation.py:506] From /home/evgenia/CourseraEnv/env/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0405 14:16:20.862070 140362287126336 deprecation.py:506] From /home/evgenia/CourseraEnv/env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 4)           868984    
_________________________________________________________________
global_average_pooling1d (Gl (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 868,997
Trainable params: 868,997
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [12]:
history = model.fit(train_data,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(test_data, y_test),
                    verbose=1)

Train on 189997 samples, validate on 10000 samples
Epoch 1/40
189997/189997 [==============================] - 6s 32us/sample - loss: 0.6867 - acc: 0.5927 - val_loss: 0.6738 - val_acc: 0.6435
Epoch 2/40
189997/189997 [==============================] - 6s 33us/sample - loss: 0.6587 - acc: 0.6569 - val_loss: 0.6445 - val_acc: 0.6803
Epoch 3/40
189997/189997 [==============================] - 7s 36us/sample - loss: 0.6307 - acc: 0.6834 - val_loss: 0.6247 - val_acc: 0.6827
Epoch 4/40
189997/189997 [==============================] - 6s 31us/sample - loss: 0.6112 - acc: 0.6940 - val_loss: 0.6132 - val_acc: 0.6889
Epoch 5/40
189997/189997 [==============================] - 6s 31us/sample - loss: 0.5982 - acc: 0.6995 - val_loss: 0.6071 - val_acc: 0.6891
Epoch 6/40
189997/189997 [==============================] - 5s 28us/sample - loss: 0.5893 - acc: 0.7032 - val_loss: 0.6037 - val_acc: 0.6931
Epoch 7/40
189997/189997 [==============================] - 5s 26us/sample - loss: 0.5829 - acc: 0.7060

In [13]:
from sklearn.metrics import mean_squared_error
y_pred = np.full(y_test.shape, y_train.mean())
mean_squared_error(y_test, y_pred)

0.2500035416432395